In [1]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv('user_actions.csv')
data.describe()

,actionId,currentUser,createdDateTime,message
count,18995,18995,18995,18995
unique,9965,15,18786,13400
top,ddaea92d-877c-4937-ae11-c55788fc7d91,lxusdzwncy,2019-09-02 10:53:00,[END] [SEARCH]: [num_results=9]
freq,2,1352,3,18


In [10]:
data.head()

,actionId,currentUser,createdDateTime,message
0,875ee856-7376-4d2b-bdb3-159696551ea9,tpoyqv,2019-10-03 14:12:23,"[END] [IMPORT]: [num_success=2, num_failed=1, ..."
1,7c592432-a361-41e9-9e1d-3b7097e74447,okiuvpr,2019-10-23 10:10:00,[START] [SEARCH]: [approved_by=renwhscyxdi]
2,9f2287f6-425a-4d5a-9381-bec13285e2fc,hojgrxwbinkfcp,2019-06-03 14:36:00,[START] [IMPORT]: [num_records=497]
3,b830e027-5f2b-41fc-bd19-9580e0f25cea,lxusdzwncy,2019-04-03 12:57:10,"[END] [APPROVE]: [num_success=2, num_failed=49..."
4,f4d9f09f-f83c-41a4-9abd-fd5c66bf1bfb,okiuvpr,2019-02-10 15:07:40,"[END] [IMPORT]: [num_success=41, num_failed=8,..."


In [7]:
data['createdDateTime'] = pd.to_datetime(data['createdDateTime'])

In [8]:
data.isna().sum()

actionId           0
currentUser        0
createdDateTime    0
message            0
dtype: int64

In [14]:
data['status'] = data['message'].apply(lambda m: m.split(']')[0][1:])
data['operation_type'] = data['message'].apply(lambda m: m.split(']')[1][1:])


0          END
1        START
2        START
3          END
4          END
         ...  
18990    START
18991      END
18992    START
18993      END
18994    START
Name: status, Length: 18995, dtype: object